In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
torch.__version__

'2.1.2'

In [73]:
#Labels
A = torch.tensor([[1,2],[3,4]],dtype=torch.float32)
B = torch.tensor([[5,6],[7,8]], dtype=torch.float32)

X = torch.cat((A,B),0)
#X = torch.stack((A,B),0)
y = torch.tensor([0,0,1,1],dtype=torch.float32)
X

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])

In [74]:
y

tensor([0., 0., 1., 1.])

In [26]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size): 
        super(SimpleNN, self).__init__()

        # layers
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

        # Activation Function
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [41]:
input_size = 2
hidden_size = 4
output_size = 2

model = SimpleNN(input_size, hidden_size, output_size)

if torch.cuda.is_available():
    model = model.cuda()

In [53]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [77]:
# convert to one-hot encoding
#y_one_hot = torch.zeros(len(y), 2)
#y_one_hot.scatter_(1, y.unsqueeze(1).to(torch.int64), 1.0)
y_one_hot = nn.functional.one_hot(y)

if torch.cuda.is_available():
    X = X.cuda()
    y_one_hot = y_one_hot.cuda()

X.device, y_one_hot.device

RuntimeError: one_hot is only applicable to index tensor.

In [66]:
print(X.dtype, y_one_hot.dtype)

torch.float32 torch.float32


In [69]:
# training
epochs=100
for epoch in range(epochs):

    optimizer.zero_grad()
    outputs=model(X)
    loss = criterion(outputs,y_one_hot)
    loss.backward()
    optimizer.step()
    if (epoch+1)%10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 10/100, Loss: 0.7550421953201294
Epoch 20/100, Loss: 0.7457253932952881
Epoch 30/100, Loss: 0.7367979288101196
Epoch 40/100, Loss: 0.72788006067276
Epoch 50/100, Loss: 0.7189001441001892
Epoch 60/100, Loss: 0.7091460227966309
Epoch 70/100, Loss: 0.6926745772361755
Epoch 80/100, Loss: 0.6849929094314575
Epoch 90/100, Loss: 0.6781398057937622
Epoch 100/100, Loss: 0.6714069843292236


# First Model

In [47]:
import torch
import torch.nn as nn

In [36]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02
X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight * X + bias

X

tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800],
        [0.2000],
        [0.2200],
        [0.2400],
        [0.2600],
        [0.2800],
        [0.3000],
        [0.3200],
        [0.3400],
        [0.3600],
        [0.3800],
        [0.4000],
        [0.4200],
        [0.4400],
        [0.4600],
        [0.4800],
        [0.5000],
        [0.5200],
        [0.5400],
        [0.5600],
        [0.5800],
        [0.6000],
        [0.6200],
        [0.6400],
        [0.6600],
        [0.6800],
        [0.7000],
        [0.7200],
        [0.7400],
        [0.7600],
        [0.7800],
        [0.8000],
        [0.8200],
        [0.8400],
        [0.8600],
        [0.8800],
        [0.9000],
        [0.9200],
        [0.9400],
        [0.9600],
        [0.9800]])

In [38]:
id = int(len(X) * 0.8)
X_train, y_train = X[:id],y[:id]
X_test, y_test = X[id:], y[id:]

In [64]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                             requires_grad=True,
                                             dtype=torch.float32))
        self.bias = nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float32))
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

In [205]:
torch.manual_seed (42)

model = LinearRegressionModel()

list(model.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [66]:
# List name Parameters
model.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [67]:
## Prediccion
with torch.inference_mode():
    y_pred = model(X_test)

In [68]:
y_pred,y_test

(tensor([[0.3982],
         [0.4049],
         [0.4116],
         [0.4184],
         [0.4251],
         [0.4318],
         [0.4386],
         [0.4453],
         [0.4520],
         [0.4588]]),
 tensor([[0.8600],
         [0.8740],
         [0.8880],
         [0.9020],
         [0.9160],
         [0.9300],
         [0.9440],
         [0.9580],
         [0.9720],
         [0.9860]]))

In [206]:
from torch.optim import Adam, SGD

loss_fn = nn.L1Loss()

# Definición de optimizadores
optim_adam = Adam(params=model.parameters(),
                 lr=0.01)
optim_sgd = SGD(params=model.parameters(),
               lr=0.01)

In [210]:
### Training Loop

epochs = 50
for epoch in range(epochs):
    # 0. Modo entrenamiento, setea todos los parametros que requieren gradiente. 
    model.train() 
    
    # 1. Forward pass
    y_pred = model(X_train)
    
    # 2. Calculo del Error 
    loss = loss_fn(y_pred,y_train)
    print(f'loss: {loss}')

    # 3. Resetea el 
    optim_adam.zero_grad()

    # 4. BackPropagation
    loss.backward()

    # 5. Calculo del Gradiente
    optim_adam.step()
    
    model.eval() # apaga el seguimiento del gradiente
    with torch.inference_mode():
        test_pred = model(X_test)
        loss_test = loss_fn(test_pred,y_test)

    if epoch % 5 == 0:
        print(f'Epoch: {epoch} / Train Loss: {loss} / Test Loss: {loss_test}')
        print(model.state_dict())

loss: 0.2711813449859619
Epoch: 0 / Train Loss: 0.2711813449859619 / Test Loss: 0.418936163187027
OrderedDict([('weights', tensor([0.3767])), ('bias', tensor([0.1688]))])
loss: 0.2572813332080841
loss: 0.2433813512325287
loss: 0.22948133945465088
loss: 0.21558134257793427
loss: 0.20168134570121765
Epoch: 5 / Train Loss: 0.20168134570121765 / Test Loss: 0.324436217546463
OrderedDict([('weights', tensor([0.4267])), ('bias', tensor([0.2188]))])
loss: 0.18778134882450104
loss: 0.17388132214546204
loss: 0.15998134016990662
loss: 0.1460813283920288
loss: 0.1321813315153122
Epoch: 10 / Train Loss: 0.1321813315153122 / Test Loss: 0.22993624210357666
OrderedDict([('weights', tensor([0.4767])), ('bias', tensor([0.2688]))])
loss: 0.11828134208917618
loss: 0.10438136011362076
loss: 0.09048136323690414
loss: 0.07658137381076813
loss: 0.06345288455486298
Epoch: 15 / Train Loss: 0.06345288455486298 / Test Loss: 0.13554081320762634
OrderedDict([('weights', tensor([0.5267])), ('bias', tensor([0.3187]))

In [63]:
y_pred = model(X_test)
y_pred

tensor([[0.5962],
        [0.6051],
        [0.6140],
        [0.6230],
        [0.6319],
        [0.6408],
        [0.6498],
        [0.6587],
        [0.6676],
        [0.6766]], grad_fn=<AddBackward0>)